# Parameters
CampañasxPeriodo=18
UltimaCampañaRolling=202818
CampañaInicioRolling=202613
Carpeta=r'C:\Users\williamtorres\Desktop\Nueva carpeta (3)\Resultado CORP\101'

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime

In [3]:
df = pd.read_csv(rf'{Carpeta}\df_NovoApp.csv')

In [4]:
df.head()

,Ce.,Material,Campaña,Fuente,Tipo de Oferta,Cantidad estimada,Descontinuación,Descripción,Grupo art.,Tipo de producto,Tipo,Grupo art..1,Crecimiento X,Crecimiento X+1,Crecimiento X+2,Crecimiento X+3,EDL
0,BO03,200113784.0,202606,Novoapp,321,485,0,ES SED ABSOLUTE PARF 100 ML,101.0,PARFUM,101,Fragancias,0.065541,0.052837,0.053188,0.050226,-
1,BO03,200113784.0,202606,Novoapp,421,1080,0,ES SED ABSOLUTE PARF 100 ML,101.0,PARFUM,101,Fragancias,0.065541,0.052837,0.053188,0.050226,-
2,BO03,200115806.0,202606,Novoapp,300,629,0,CZ MISS SEXY RADIANT COL 200ML,101.0,COLONIA FAMILIAR,101,Fragancias,0.065541,0.052837,0.053188,0.050226,-
3,BO03,200115811.0,202606,Novoapp,300,1067,0,CZ B&B ME HE EDP 90ML,101.0,EAU DE PARFUM,101,Fragancias,0.065541,0.052837,0.053188,0.050226,-
4,BO03,200115814.0,202606,Novoapp,300,1921,0,CZ B&B ME HI EDP 90ML,101.0,COLONIA,101,Fragancias,0.065541,0.052837,0.053188,0.050226,-


In [5]:
# Crear tabla dinámica
tabla_pivot = pd.pivot_table(
    df,
    index=["Ce.", "Grupo art.", "Crecimiento X+2", "Crecimiento X+3", "Descontinuación", "Material"], # filas
    columns="Campaña",   # columnas
    values="Cantidad estimada",  # la métrica que quieres sumar/promediar
    aggfunc="sum",       # puede ser sum, mean, count, etc.
    fill_value=0
)

In [6]:
tabla_pivot

Campaña                                                                      202606  \
Ce.  Grupo art. Crecimiento X+2 Crecimiento X+3 Descontinuación Material              
BO03 101.0      0.053188        0.050226        0               200113553.0       0   
                                                                200113784.0    1565   
                                                                200115806.0     629   
                                                                200115811.0    1067   
                                                                200115814.0    1921   
...                                                                             ...   
SV13 101.0      0.053188        0.050226        0               200120632.0       0   
                                                                200120788.0     354   
                                                                200120953.0       0   
                                                                200120956.0       0   
                                                                200121381.0     900   

Campaña                                                                      202607  \
Ce.  Grupo art. Crecimiento X+2 Crecimiento X+3 Descontinuación Material              
BO03 101.0      0.053188        0.050226        0               200113553.0       0   
                                                                200113784.0     247   
                                                                200115806.0    1596   
                                                                200115811.0     620   
                                                                200115814.0     759   
...                                                                             ...   
SV13 101.0      0.053188        0.050226        0               200120632.0       0   
                                                                200120788.0     354   
                                                                200120953.0       0   
                                                                200120956.0       0   
                                                                200121381.0       0   

Campaña                                                                      202608  \
Ce.  Grupo art. Crecimiento X+2 Crecimiento X+3 Descontinuación Material              
BO03 101.0      0.053188        0.050226        0               200113553.0       0   
                                                                200113784.0     621   
                                                                200115806.0     679   
                                                                200115811.0       0   
                                                                200115814.0       0   
...                                                                             ...   
SV13 101.0      0.053188        0.050226        0               200120632.0       0   
                                                                200120788.0     354   
                                                                200120953.0    1286   
                                                                200120956.0       0   
                                                                200121381.0       0   

Campaña                                                                      202609  \
Ce.  Grupo art. Crecimiento X+2 Crecimiento X+3 Descontinuación Material              
BO03 101.0      0.053188        0.050226        0               200113553.0       0   
                                                                200113784.0     477   
                                                                200115806.0    2113   
                                                                200115811.0       0   
                                                                200115814.0       

In [7]:
def generar_campañas(inicio, fin):
    """
    Genera lista de campañas desde 'inicio' hasta 'fin'
    considerando que cada año solo llega hasta 18,
    luego pasa al siguiente año con 01.
    """
    campañas = []
    año = int(str(inicio)[:4])
    periodo = int(str(inicio)[-2:])
    año_fin = int(str(fin)[:4])
    periodo_fin = int(str(fin)[-2:])

    while (año < año_fin) or (año == año_fin and periodo <= periodo_fin):
        campañas.append(int(f"{año}{periodo:02d}"))
        periodo += 1
        if periodo > CampañasxPeriodo:   # al pasar de 18 reinicia
            periodo = 1
            año += 1
    return campañas


In [8]:
# Generar campañas desde la mínima hasta 202818
campaña_min = int(df["Campaña"].min())
campañas_completas = generar_campañas(campaña_min, UltimaCampañaRolling)

# Reindexar columnas
tabla_pivot = tabla_pivot.reindex(columns=campañas_completas, fill_value=0)

# Tabla plana opcional
tabla_pivot = tabla_pivot.reset_index()

In [9]:
tabla_pivot.head()

Campaña,Ce.,Grupo art.,Crecimiento X+2,Crecimiento X+3,Descontinuación,Material,202606,202607,202608,202609,...,202809,202810,202811,202812,202813,202814,202815,202816,202817,202818
0,BO03,101.0,0.053188,0.050226,0,200113553.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,BO03,101.0,0.053188,0.050226,0,200113784.0,1565,247,621,477,...,0,0,0,0,0,0,0,0,0,0
2,BO03,101.0,0.053188,0.050226,0,200115806.0,629,1596,679,2113,...,0,0,0,0,0,0,0,0,0,0
3,BO03,101.0,0.053188,0.050226,0,200115811.0,1067,620,0,0,...,0,0,0,0,0,0,0,0,0,0
4,BO03,101.0,0.053188,0.050226,0,200115814.0,1921,759,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
# Buscar la última campaña con valor > 0 en cada fila
tabla_pivot["ÚltimoCampañaNovoApp"] = tabla_pivot.apply(
    lambda row: max([col for col in tabla_pivot.columns if isinstance(col, int) and row[col] > 0], default=None),
    axis=1
)

In [11]:
tabla_pivot_CargadoSAP=tabla_pivot.copy()

In [12]:
tabla_pivot

Campaña,Ce.,Grupo art.,Crecimiento X+2,Crecimiento X+3,Descontinuación,Material,202606,202607,202608,202609,...,202810,202811,202812,202813,202814,202815,202816,202817,202818,ÚltimoCampañaNovoApp
0,BO03,101.0,0.053188,0.050226,0,200113553.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,202709.0
1,BO03,101.0,0.053188,0.050226,0,200113784.0,1565,247,621,477,...,0,0,0,0,0,0,0,0,0,202610.0
2,BO03,101.0,0.053188,0.050226,0,200115806.0,629,1596,679,2113,...,0,0,0,0,0,0,0,0,0,202611.0
3,BO03,101.0,0.053188,0.050226,0,200115811.0,1067,620,0,0,...,0,0,0,0,0,0,0,0,0,202607.0
4,BO03,101.0,0.053188,0.050226,0,200115814.0,1921,759,0,0,...,0,0,0,0,0,0,0,0,0,202607.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
523,SV13,101.0,0.053188,0.050226,0,200120632.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,202704.0
524,SV13,101.0,0.053188,0.050226,0,200120788.0,354,354,354,354,...,0,0,0,0,0,0,0,0,0,202618.0
525,SV13,101.0,0.053188,0.050226,0,200120953.0,0,0,1286,0,...,0,0,0,0,0,0,0,0,0,202709.0
526,SV13,101.0,0.053188,0.050226,0,200120956.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,202718.0


In [13]:
def siguiente_campaña(camp):
    """Dada una campaña tipo 202615, devuelve la siguiente (202616 o 202701 si era 18)."""
    camp = int(camp)
    año = camp // 100
    num = camp % 100
    if num == 18:
        año += 1
        num = 1
    else:
        num += 1
    return int(f"{año}{num:02d}")


for idx_row, row in tabla_pivot.iterrows():
    ultima = row["ÚltimoCampañaNovoApp"]
    descontinuacion = row["Descontinuación"]

    if pd.isna(descontinuacion) or descontinuacion == 0:
        descontinuacion = UltimaCampañaRolling
    
    if pd.isna(ultima):
        continue
    
    # siguiente campaña al último estimado
    siguiente = siguiente_campaña(ultima)
    
    # inicio: mayor entre 202612 y esa siguiente
    inicio = max(CampañaInicioRolling, siguiente)

    # si no está en campañas_completas (por ejemplo 202619), saltamos al próximo válido
    while inicio not in campañas_completas and inicio < max(campañas_completas):
        inicio = siguiente_campaña(inicio)

    if inicio not in campañas_completas:
        continue  # nada que proyectar

    start_index = campañas_completas.index(inicio)

    # proyectar desde inicio hasta el final del horizonte
    for j in range(start_index, len(campañas_completas)):
        camp = campañas_completas[j]
        if camp > descontinuacion:  # 🚫 parar si pasa la campaña de descontinuación
            break
        prev_3 = campañas_completas[start_index-3:start_index]   # las 3 campañas anteriores en la lista
        tabla_pivot.at[idx_row, camp] = row[prev_3].mean()


C:\Users\williamtorres\AppData\Local\Temp\ipykernel_14452\258172307.py:45: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '968.6666666666666' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  tabla_pivot.at[idx_row, camp] = row[prev_3].mean()
C:\Users\williamtorres\AppData\Local\Temp\ipykernel_14452\258172307.py:45: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '968.6666666666666' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  tabla_pivot.at[idx_row, camp] = row[prev_3].mean()
C:\Users\williamtorres\AppData\Local\Temp\ipykernel_14452\258172307.py:45: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '968.6666666666666' has dtype incompatible with int64, please explicitly cast to a compatible dtype firs

In [14]:
tabla_pivot

Campaña,Ce.,Grupo art.,Crecimiento X+2,Crecimiento X+3,Descontinuación,Material,202606,202607,202608,202609,...,202810,202811,202812,202813,202814,202815,202816,202817,202818,ÚltimoCampañaNovoApp
0,BO03,101.0,0.053188,0.050226,0,200113553.0,0,0.0,0.0,0.0,...,968.666667,968.666667,968.666667,968.666667,968.666667,968.666667,968.666667,968.666667,968.666667,202709.0
1,BO03,101.0,0.053188,0.050226,0,200113784.0,1565,247.0,621.0,477.0,...,976.333333,976.333333,976.333333,976.333333,976.333333,976.333333,976.333333,976.333333,976.333333,202610.0
2,BO03,101.0,0.053188,0.050226,0,200115806.0,629,1596.0,679.0,2113.0,...,1261.000000,1261.000000,1261.000000,1261.000000,1261.000000,1261.000000,1261.000000,1261.000000,1261.000000,202611.0
3,BO03,101.0,0.053188,0.050226,0,200115811.0,1067,620.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,202607.0
4,BO03,101.0,0.053188,0.050226,0,200115814.0,1921,759.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,202607.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
523,SV13,101.0,0.053188,0.050226,0,200120632.0,0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,202704.0
524,SV13,101.0,0.053188,0.050226,0,200120788.0,354,354.0,354.0,354.0,...,354.000000,354.000000,354.000000,354.000000,354.000000,354.000000,354.000000,354.000000,354.000000,202618.0
525,SV13,101.0,0.053188,0.050226,0,200120953.0,0,0.0,1286.0,0.0,...,3366.000000,3366.000000,3366.000000,3366.000000,3366.000000,3366.000000,3366.000000,3366.000000,3366.000000,202709.0
526,SV13,101.0,0.053188,0.050226,0,200120956.0,0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,202718.0


In [15]:
año_base = datetime.today().year

for idx_row, row in tabla_pivot.iterrows():
    crec_x2 = row["Crecimiento X+2"]  if not pd.isna(row["Crecimiento X+2"]) else 0
    crec_x3 = row["Crecimiento X+3"]  if not pd.isna(row["Crecimiento X+3"]) else 0

    for camp in campañas_completas:
        año = int(str(camp)[:4])  # los 4 primeros dígitos de la campaña

        if año == año_base + 2:
            tabla_pivot.at[idx_row, camp] = tabla_pivot.at[idx_row, camp] * (1 + crec_x2)

        elif año == año_base + 3:
            tabla_pivot.at[idx_row, camp] = tabla_pivot.at[idx_row, camp] * (1 + crec_x3+crec_x2)


In [16]:
# Detectar columnas que son campañas (numéricas tipo 202501, 202502, etc.)
camp_cols = [c for c in tabla_pivot.columns if str(c).isdigit()]
tabla_pivot = tabla_pivot.fillna(0)
# Redondear hacia arriba sin decimales
tabla_pivot[camp_cols] = np.ceil(tabla_pivot[camp_cols]).astype(int)

In [17]:
tabla_pivot

Campaña,Ce.,Grupo art.,Crecimiento X+2,Crecimiento X+3,Descontinuación,Material,202606,202607,202608,202609,...,202810,202811,202812,202813,202814,202815,202816,202817,202818,ÚltimoCampañaNovoApp
0,BO03,101.0,0.053188,0.050226,0,200113553.0,0,0,0,0,...,1069,1069,1069,1069,1069,1069,1069,1069,1069,202709.0
1,BO03,101.0,0.053188,0.050226,0,200113784.0,1565,247,621,477,...,1078,1078,1078,1078,1078,1078,1078,1078,1078,202610.0
2,BO03,101.0,0.053188,0.050226,0,200115806.0,629,1596,679,2113,...,1392,1392,1392,1392,1392,1392,1392,1392,1392,202611.0
3,BO03,101.0,0.053188,0.050226,0,200115811.0,1067,620,0,0,...,0,0,0,0,0,0,0,0,0,202607.0
4,BO03,101.0,0.053188,0.050226,0,200115814.0,1921,759,0,0,...,0,0,0,0,0,0,0,0,0,202607.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
523,SV13,101.0,0.053188,0.050226,0,200120632.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,202704.0
524,SV13,101.0,0.053188,0.050226,0,200120788.0,354,354,354,354,...,391,391,391,391,391,391,391,391,391,202618.0
525,SV13,101.0,0.053188,0.050226,0,200120953.0,0,0,1286,0,...,3715,3715,3715,3715,3715,3715,3715,3715,3715,202709.0
526,SV13,101.0,0.053188,0.050226,0,200120956.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,202718.0


In [18]:
# Identificar columnas de campañas (numéricas tipo 202501, 202502, etc.)
camp_cols = [c for c in tabla_pivot.columns if str(c).isdigit()]

# Copia base
tabla_proyecciones = tabla_pivot.copy()

for idx_row, row in tabla_pivot.iterrows():
    ultima = int(row["ÚltimoCampañaNovoApp"])
    # Todo lo que sea <= a la última campaña lo borramos (NaN) → solo dejamos proyecciones
    for col in camp_cols:
        if int(col) <= ultima:
            tabla_proyecciones.at[idx_row, col] = np.nan


In [19]:
# Identificar columnas de campañas (numéricas tipo 202501, 202502, etc.)
camp_cols = [c for c in tabla_pivot.columns if str(c).isdigit()]

# Copia base
tabla_proyecciones_Sistema = tabla_pivot.copy()



In [20]:
tabla_proyecciones

Campaña,Ce.,Grupo art.,Crecimiento X+2,Crecimiento X+3,Descontinuación,Material,202606,202607,202608,202609,...,202810,202811,202812,202813,202814,202815,202816,202817,202818,ÚltimoCampañaNovoApp
0,BO03,101.0,0.053188,0.050226,0,200113553.0,NaN,NaN,NaN,NaN,...,1069,1069,1069,1069,1069,1069,1069,1069,1069,202709.0
1,BO03,101.0,0.053188,0.050226,0,200113784.0,NaN,NaN,NaN,NaN,...,1078,1078,1078,1078,1078,1078,1078,1078,1078,202610.0
2,BO03,101.0,0.053188,0.050226,0,200115806.0,NaN,NaN,NaN,NaN,...,1392,1392,1392,1392,1392,1392,1392,1392,1392,202611.0
3,BO03,101.0,0.053188,0.050226,0,200115811.0,NaN,NaN,0.0,0.0,...,0,0,0,0,0,0,0,0,0,202607.0
4,BO03,101.0,0.053188,0.050226,0,200115814.0,NaN,NaN,0.0,0.0,...,0,0,0,0,0,0,0,0,0,202607.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
523,SV13,101.0,0.053188,0.050226,0,200120632.0,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,202704.0
524,SV13,101.0,0.053188,0.050226,0,200120788.0,NaN,NaN,NaN,NaN,...,391,391,391,391,391,391,391,391,391,202618.0
525,SV13,101.0,0.053188,0.050226,0,200120953.0,NaN,NaN,NaN,NaN,...,3715,3715,3715,3715,3715,3715,3715,3715,3715,202709.0
526,SV13,101.0,0.053188,0.050226,0,200120956.0,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,202718.0


In [21]:
tabla_proyecciones.to_excel(
    rf"{Carpeta}\ProyecionNovoAPP.xlsx", 
    sheet_name="Proyeccion", 
    index=False
)

In [22]:
# Detectar columnas que son campañas (numéricas tipo 202501, 202612, etc.)
camp_cols = [c for c in tabla_proyecciones.columns if str(c).isdigit()]

# Filtrar solo las columnas de campañas >= 202612
camp_cols_filtradas = [c for c in camp_cols if int(c) >= CampañaInicioRolling]

# Mantener columnas fijas + las filtradas
otras_cols = [c for c in tabla_proyecciones.columns if c not in camp_cols]
tabla_filtrada = tabla_proyecciones[otras_cols + camp_cols_filtradas]


In [23]:
tabla_filtrada = tabla_filtrada.fillna(0)
tabla_filtrada

Campaña,Ce.,Grupo art.,Crecimiento X+2,Crecimiento X+3,Descontinuación,Material,ÚltimoCampañaNovoApp,202613,202614,202615,...,202809,202810,202811,202812,202813,202814,202815,202816,202817,202818
0,BO03,101.0,0.053188,0.050226,0,200113553.0,202709.0,0.0,0.0,0.0,...,1069,1069,1069,1069,1069,1069,1069,1069,1069,1069
1,BO03,101.0,0.053188,0.050226,0,200113784.0,202610.0,977.0,977.0,977.0,...,1078,1078,1078,1078,1078,1078,1078,1078,1078,1078
2,BO03,101.0,0.053188,0.050226,0,200115806.0,202611.0,1261.0,1261.0,1261.0,...,1392,1392,1392,1392,1392,1392,1392,1392,1392,1392
3,BO03,101.0,0.053188,0.050226,0,200115811.0,202607.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,BO03,101.0,0.053188,0.050226,0,200115814.0,202607.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
523,SV13,101.0,0.053188,0.050226,0,200120632.0,202704.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
524,SV13,101.0,0.053188,0.050226,0,200120788.0,202618.0,0.0,0.0,0.0,...,391,391,391,391,391,391,391,391,391,391
525,SV13,101.0,0.053188,0.050226,0,200120953.0,202709.0,0.0,0.0,0.0,...,3715,3715,3715,3715,3715,3715,3715,3715,3715,3715
526,SV13,101.0,0.053188,0.050226,0,200120956.0,202718.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
tabla_filtrada.columns

Index([                 'Ce.',           'Grupo art.',      'Crecimiento X+2',
            'Crecimiento X+3',      'Descontinuación',             'Material',
       'ÚltimoCampañaNovoApp',                 202613,                 202614,
                       202615,                 202616,                 202617,
                       202618,                 202701,                 202702,
                       202703,                 202704,                 202705,
                       202706,                 202707,                 202708,
                       202709,                 202710,                 202711,
                       202712,                 202713,                 202714,
                       202715,                 202716,                 202717,
                       202718,                 202801,                 202802,
                       202803,                 202804,                 202805,
                       202806,                 20280

In [25]:
tabla_filtrada= tabla_filtrada.drop(columns=['Grupo art.','Crecimiento X+2',
            'Crecimiento X+3',      'Descontinuación', 'ÚltimoCampañaNovoApp'])
tabla_filtrada.head()

Campaña,Ce.,Material,202613,202614,202615,202616,202617,202618,202701,202702,...,202809,202810,202811,202812,202813,202814,202815,202816,202817,202818
0,BO03,200113553.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1069,1069,1069,1069,1069,1069,1069,1069,1069,1069
1,BO03,200113784.0,977.0,977.0,977.0,977.0,977.0,977.0,1029.0,1029.0,...,1078,1078,1078,1078,1078,1078,1078,1078,1078,1078
2,BO03,200115806.0,1261.0,1261.0,1261.0,1261.0,1261.0,1261.0,1329.0,1329.0,...,1392,1392,1392,1392,1392,1392,1392,1392,1392,1392
3,BO03,200115811.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,BO03,200115814.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
tabla_larga = tabla_filtrada.melt(
    id_vars=["Ce.", 'Material'],  
    var_name="Campaña",
    value_name="Estimado"
)

In [27]:
tabla_larga=tabla_larga[tabla_larga["Estimado"]>0]
tabla_larga["TO"] = "'000"
tabla_larga["Estimado"] = tabla_larga["Estimado"].astype(int)
tabla_larga["Material"] = tabla_larga["Material"].astype(int)
tabla_larga.columns=["Ce.",'Cod', 'Campaña', 'Estimado', "TO" ]
tabla_larga.head()

,Ce.,Cod,Campaña,Estimado,TO
1,BO03,200113784,202613,977,'000
2,BO03,200115806,202613,1261,'000
7,BO03,200116520,202613,363,'000
10,BO03,200117098,202613,730,'000
14,BO03,200117127,202613,1372,'000


In [28]:
tabla_larga=tabla_larga[["Ce.", 'Campaña', 'Cod', "TO", 'Estimado']]

In [29]:
tabla_larga

,Ce.,Campaña,Cod,TO,Estimado
1,BO03,202613,200113784,'000,977
2,BO03,202613,200115806,'000,1261
7,BO03,202613,200116520,'000,363
10,BO03,202613,200117098,'000,730
14,BO03,202613,200117127,'000,1372
...,...,...,...,...,...
22169,SV13,202818,200120475,'000,181
22170,SV13,202818,200120587,'000,1208
22172,SV13,202818,200120788,'000,391
22173,SV13,202818,200120953,'000,3715


In [30]:
tabla_larga.to_excel(
    rf"{Carpeta}\CargaNovoApp.xlsx", 
    sheet_name="Proyeccion", 
    index=False
)

In [31]:
tabla_larga["Año"] = tabla_larga["Campaña"].astype(str).str[:4].astype(int)
tabla_larga["campaña"]   = tabla_larga["Campaña"].astype(str).str[-2:].astype(int)

In [32]:
tabla_larga

,Ce.,Campaña,Cod,TO,Estimado,Año,campaña
1,BO03,202613,200113784,'000,977,2026,13
2,BO03,202613,200115806,'000,1261,2026,13
7,BO03,202613,200116520,'000,363,2026,13
10,BO03,202613,200117098,'000,730,2026,13
14,BO03,202613,200117127,'000,1372,2026,13
...,...,...,...,...,...,...,...
22169,SV13,202818,200120475,'000,181,2028,18
22170,SV13,202818,200120587,'000,1208,2028,18
22172,SV13,202818,200120788,'000,391,2028,18
22173,SV13,202818,200120953,'000,3715,2028,18


In [33]:
tabla_grouped = (
    tabla_larga
    .groupby(["Año", "campaña"], as_index=False)["Estimado"]
    .sum()
)

In [34]:
tabla_pivot = tabla_larga.pivot_table(
    index="campaña",        # filas
    columns="Año",    # columnas
    values="Estimado",      # valores
    aggfunc="sum",    # sumar
    fill_value=0      # reemplazar NaN con 0
)


# Quitar el nombre del índice de columnas ("Año")
tabla_pivot.columns.name = None

# Opcional: resetear índice si no quieres que "C" quede como índice
tabla_pivot = tabla_pivot.reset_index()

## Estimados cargados por Rolling

In [35]:
tabla_pivot

,campaña,2026,2027,2028
0,1,0,318659,764343
1,2,0,327682,764343
2,3,0,363551,764343
3,4,0,386052,764343
4,5,0,427842,764343
5,6,0,427842,764343
6,7,0,488739,764343
7,8,0,488739,764343
8,9,0,491864,764343
9,10,0,691157,764343


In [36]:
tabla_pivot.to_csv(rf"{Carpeta}\TotalNovoAPPRolling.csv", index=False, encoding="utf-8-sig")

## Total NovoApp con propagación

In [37]:
tabla_proyecciones=tabla_proyecciones_Sistema

In [38]:
# Detectar columnas que son campañas (numéricas tipo 202501, 202612, etc.)
camp_cols = [c for c in tabla_proyecciones.columns if str(c).isdigit()]

# Filtrar solo las columnas de campañas >= 202612
camp_cols_filtradas = [c for c in camp_cols if int(c) >= CampañaInicioRolling]

# Mantener columnas fijas + las filtradas
otras_cols = [c for c in tabla_proyecciones.columns if c not in camp_cols]
tabla_filtrada = tabla_proyecciones[otras_cols + camp_cols_filtradas]
tabla_filtrada = tabla_filtrada.fillna(0)
tabla_filtrada
tabla_filtrada= tabla_filtrada.drop(columns=['Grupo art.','Crecimiento X+2',
            'Crecimiento X+3',      'Descontinuación', 'ÚltimoCampañaNovoApp'])
tabla_larga = tabla_filtrada.melt(
    id_vars=["Ce.", 'Material'],  
    var_name="Campaña",
    value_name="Estimado"
)
tabla_larga=tabla_larga[tabla_larga["Estimado"]>0]
tabla_larga["TO"] = "'000"
tabla_larga["Estimado"] = tabla_larga["Estimado"].astype(int)
tabla_larga["Material"] = tabla_larga["Material"].astype(int)
tabla_larga.columns=["Ce.",'Cod', 'Campaña', 'Estimado', "TO" ]
tabla_larga=tabla_larga[["Ce.", 'Campaña', 'Cod', "TO", 'Estimado']]
tabla_larga["Año"] = tabla_larga["Campaña"].astype(str).str[:4].astype(int)
tabla_larga["campaña"]   = tabla_larga["Campaña"].astype(str).str[-2:].astype(int)
tabla_grouped = (
    tabla_larga
    .groupby(["Año", "campaña"], as_index=False)["Estimado"]
    .sum()
)
tabla_pivot = tabla_larga.pivot_table(
    index="campaña",        # filas
    columns="Año",    # columnas
    values="Estimado",      # valores
    aggfunc="sum",    # sumar
    fill_value=0      # reemplazar NaN con 0
)


# Quitar el nombre del índice de columnas ("Año")
tabla_pivot.columns.name = None

# Opcional: resetear índice si no quieres que "C" quede como índice
tabla_pivot = tabla_pivot.reset_index()

In [39]:
tabla_pivot

,campaña,2026,2027,2028
0,1,0,729970,764343
1,2,0,729970,764343
2,3,0,729970,764343
3,4,0,729560,764343
4,5,0,729560,764343
5,6,0,729560,764343
6,7,0,729560,764343
7,8,0,729560,764343
8,9,0,729560,764343
9,10,0,729560,764343


In [40]:
tabla_pivot.to_csv(rf"{Carpeta}\TotalNovoAPP.csv", index=False, encoding="utf-8-sig")

## Cargados en el sistema

In [41]:
tabla_pivot_CargadoSAP

Campaña,Ce.,Grupo art.,Crecimiento X+2,Crecimiento X+3,Descontinuación,Material,202606,202607,202608,202609,...,202810,202811,202812,202813,202814,202815,202816,202817,202818,ÚltimoCampañaNovoApp
0,BO03,101.0,0.053188,0.050226,0,200113553.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,202709.0
1,BO03,101.0,0.053188,0.050226,0,200113784.0,1565,247,621,477,...,0,0,0,0,0,0,0,0,0,202610.0
2,BO03,101.0,0.053188,0.050226,0,200115806.0,629,1596,679,2113,...,0,0,0,0,0,0,0,0,0,202611.0
3,BO03,101.0,0.053188,0.050226,0,200115811.0,1067,620,0,0,...,0,0,0,0,0,0,0,0,0,202607.0
4,BO03,101.0,0.053188,0.050226,0,200115814.0,1921,759,0,0,...,0,0,0,0,0,0,0,0,0,202607.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
523,SV13,101.0,0.053188,0.050226,0,200120632.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,202704.0
524,SV13,101.0,0.053188,0.050226,0,200120788.0,354,354,354,354,...,0,0,0,0,0,0,0,0,0,202618.0
525,SV13,101.0,0.053188,0.050226,0,200120953.0,0,0,1286,0,...,0,0,0,0,0,0,0,0,0,202709.0
526,SV13,101.0,0.053188,0.050226,0,200120956.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,202718.0


In [42]:
tabla_proyecciones=tabla_pivot_CargadoSAP

In [43]:
# Detectar columnas que son campañas (numéricas tipo 202501, 202612, etc.)
camp_cols = [c for c in tabla_proyecciones.columns if str(c).isdigit()]

# Filtrar solo las columnas de campañas >= 202612
camp_cols_filtradas = [c for c in camp_cols if int(c) >= CampañaInicioRolling]

# Mantener columnas fijas + las filtradas
otras_cols = [c for c in tabla_proyecciones.columns if c not in camp_cols]
tabla_filtrada = tabla_proyecciones[otras_cols + camp_cols_filtradas]
tabla_filtrada = tabla_filtrada.fillna(0)
tabla_filtrada
tabla_filtrada= tabla_filtrada.drop(columns=['Grupo art.','Crecimiento X+2',
            'Crecimiento X+3',      'Descontinuación', 'ÚltimoCampañaNovoApp'])
tabla_larga = tabla_filtrada.melt(
    id_vars=["Ce.", 'Material'],  
    var_name="Campaña",
    value_name="Estimado"
)
#tabla_larga=tabla_larga[tabla_larga["Estimado"]>0]
tabla_larga["TO"] = "'000"
tabla_larga["Estimado"] = tabla_larga["Estimado"].astype(int)
tabla_larga["Material"] = tabla_larga["Material"].astype(int)
tabla_larga.columns=["Ce.",'Cod', 'Campaña', 'Estimado', "TO" ]
tabla_larga=tabla_larga[["Ce.", 'Campaña', 'Cod', "TO", 'Estimado']]
tabla_larga["Año"] = tabla_larga["Campaña"].astype(str).str[:4].astype(int)
tabla_larga["campaña"]   = tabla_larga["Campaña"].astype(str).str[-2:].astype(int)
tabla_grouped = (
    tabla_larga
    .groupby(["Año", "campaña"], as_index=False)["Estimado"]
    .sum()
)
tabla_pivot = tabla_larga.pivot_table(
    index="campaña",        # filas
    columns="Año",    # columnas
    values="Estimado",      # valores
    aggfunc="sum",    # sumar
    fill_value=0      # reemplazar NaN con 0
)


# Quitar el nombre del índice de columnas ("Año")
tabla_pivot.columns.name = None

# Opcional: resetear índice si no quieres que "C" quede como índice
tabla_pivot = tabla_pivot.reset_index()

In [44]:
tabla_pivot

,campaña,2026,2027,2028
0,1,0,353611,4505
1,2,0,314290,0
2,3,0,333004,0
3,4,0,221172,0
4,5,0,146189,0
5,6,0,170325,0
6,7,0,87108,0
7,8,0,89328,0
8,9,0,86627,0
9,10,0,30076,0


In [45]:
tabla_pivot.to_csv(rf"{Carpeta}\TotalNovoAPPSistema.csv", index=False, encoding="utf-8-sig")